NAVIGATION ROS STACK INTRO

ITESM CCM 
Oscar Fuentes


Para correr este notebook debe tenerse familiaridad con términos de ROS como tópicos, listeners, publishers, mensajes etc.


In [2]:
import math
import numpy as np
import rospy
import time
from sensor_msgs.msg import LaserScan
from geometry_msgs.msg import Twist


Introducimos ahora dos nuevos tipos de mensajes relevantes. Imagen  y nube de puntos, Ambos mensajes propios de ros, 
se muestran algunas librerías que hacen más fácil su utilización en Python
https://www.ros.org/
http://wiki.ros.org/ros_numpy
http://wiki.ros.org/cv_bridge


In [3]:
import ros_numpy
import tf


In [4]:
rospy.init_node('navigation_notebook_node') 
base_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)

In [5]:
from nav_msgs.msg import Path

In [6]:
import tf
import tf2_ros
tfBuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfBuffer)
def get_coords ():
    t = tfBuffer.lookup_transform('map', 'base_link', rospy.Time())
    quat=t.transform.rotation
    _,_,yaw= tf.transformations.euler_from_quaternion(np.asarray((quat.x,quat.y,quat.z,quat.w)))
    #print (t.transform.translation.x,t.transform.translation.y,yaw)
    
    return np.asarray((t.transform.translation.x,t.transform.translation.y,yaw))


In [18]:
xyz=get_coords()
xyz

array([-0.11800262, -0.10362133,  0.14511519])

NameError: name 'aaa' is not defined

In [21]:
from geometry_msgs.msg   import  PoseStamped
from nav_msgs.msg import Path
pose = PoseStamped()
path= Path()
#secs,nsecs= rospy.Time.now().to_sec(),rospy.Time.now().to_nsec()
path.header.frame_id='odom'
#path.header.stamp.secs=secs
#path.header.stamp.nsecs=nsecs
pub_path= rospy.Publisher("/path_tst", Path, queue_size=10)
for i in range (25):
    
    #pse.header.stamp.nsecs=56000000
    pose=PoseStamped()
    pose.header.frame_id='base_link'
    pose.header.stamp= path.header.stamp
    pose.pose.position.x=1
    pose.pose.position.y=0.0+i*2
    path.poses.append(pose)
pub_path.publish(path)


In [22]:
cents_path=[]
for pose in path.poses:
    quat=np.asarray((pose.pose.orientation.w,pose.pose.orientation.x,pose.pose.orientation.y,pose.pose.orientation.z))
    _,_,yaw= tf.transformations.euler_from_quaternion(quat)
    cents_path.append((pose.pose.position.x,pose.pose.position.y,yaw))
cc= np.asarray(cents_path)

In [23]:

cc

array([[ 1.,  0.,  0.],
       [ 1.,  2.,  0.],
       [ 1.,  4.,  0.],
       [ 1.,  6.,  0.],
       [ 1.,  8.,  0.],
       [ 1., 10.,  0.],
       [ 1., 12.,  0.],
       [ 1., 14.,  0.],
       [ 1., 16.,  0.],
       [ 1., 18.,  0.],
       [ 1., 20.,  0.],
       [ 1., 22.,  0.],
       [ 1., 24.,  0.],
       [ 1., 26.,  0.],
       [ 1., 28.,  0.],
       [ 1., 30.,  0.],
       [ 1., 32.,  0.],
       [ 1., 34.,  0.],
       [ 1., 36.,  0.],
       [ 1., 38.,  0.],
       [ 1., 40.,  0.],
       [ 1., 42.,  0.],
       [ 1., 44.,  0.],
       [ 1., 46.,  0.],
       [ 1., 48.,  0.]])

In [25]:
def get_signed_e():
    
    xyth=get_coords()#Cordenadas robot tiempo t
    closest_index=    np.linalg.norm((cc-xyth.T),axis=0).argmin()  # punto de la trayectoria más cercano
    closest_index_1= closest_index + 1                             #punto siguiente al más cercano
    p1,p2= cc[closest_index,:2],cc[closest_index_1,:2]              #Vector trayectoria ( entre puntos anteriores)
    cc[closest_index],cc[closest_index_1]
    p3= xyth[:2]                                                     #Punto robot para vector error 
    d = np.linalg.norm(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1)   # error producto cruz normalizado
    #print(d,'d')
    pto_rob=p2-xyth[:2]
    print(pto_rob)
    if (pto_rob[0]>0):sign_e=-1*d
    else:sign_e=d
    return sign_e    #Signed point to line distance

In [26]:

def move_base_vel(vx, vy, vw):
    twist = Twist()
    twist.linear.x = vx
    twist.linear.y = vy
    twist.angular.z = vw 
    base_vel_pub.publish(twist)

def move_base(x,y,yaw,timeout=5):
    start_time = rospy.Time.now().to_sec()
    while rospy.Time.now().to_sec() - start_time < timeout:  
        move_base_vel(x, y, yaw)
    move_base_vel(0,0, 0)

In [27]:
move_base(0,0,0)

In [28]:
pose = PoseStamped()
path= Path()
#secs,nsecs= rospy.Time.now().to_sec(),rospy.Time.now().to_nsec()
path.header.frame_id='base_link'
#path.header.stamp.secs=secs
#path.header.stamp.nsecs=nsecs
pub_path= rospy.Publisher("/path_tst", Path, queue_size=10)
for i in range (25):
    
    #pse.header.stamp.nsecs=56000000
    pose=PoseStamped()
    pose.header.frame_id='odom'
    pose.header.stamp= path.header.stamp
    pose.pose.position.x=1+i*2
    pose.pose.position.y=0.0
    path.poses.append(pose)
pub_path.publish(path)


In [30]:
for i in range(200):
    #xyz=get_coords()
    #closest_index=    np.linalg.norm((cc-xyz.T),axis=0).argmin()
    #closest_index_1= closest_index + 1
    #p1,p2= cc[closest_index,:2],cc[closest_index_1,:2]
    #p3= get_coords()[:2]
    #d = np.linalg.norm(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1)
    #error =d    
    error=get_signed_e()
    #PROPORTIONAL CONTROL
    # SINGLE STEP
    Kp=0.1
    vel_yaw= error * Kp
    print(f' step number {i}error signed{error} ,velyaw,{vel_yaw}')
    #move_base_vel(0.15,0.0, vel_yaw)
    move_base(0.21,0.0, vel_yaw, 0.51)
move_base_vel(0,0, 0)


[1.5971024  3.18280032]
 step number 0error signed-1.5971023968226983 ,velyaw,-0.15971023968226983


         at line 278 in /tmp/binarydeb/ros-noetic-tf2-0.7.6/src/buffer_core.cpp


[1.7158116  3.21887431]
 step number 1error signed-1.7158116004971093 ,velyaw,-0.17158116004971094
[1.81635997 3.25151773]
 step number 2error signed-1.816359974622813 ,velyaw,-0.18163599746228132
[1.91275581 3.28580482]
 step number 3error signed-1.9127558056502019 ,velyaw,-0.1912755805650202
[2.01764118 3.29915718]
 step number 4error signed-2.0176411830803627 ,velyaw,-0.20176411830803628
[2.13000063 3.31311076]
 step number 5error signed-2.130000628934917 ,velyaw,-0.21300006289349172
[2.23548946 3.30605437]
 step number 6error signed-2.2354894622672288 ,velyaw,-0.2235489462267229


         at line 278 in /tmp/binarydeb/ros-noetic-tf2-0.7.6/src/buffer_core.cpp


[2.32184058 3.28049639]
 step number 7error signed-2.321840579540912 ,velyaw,-0.23218405795409122
[2.4253939  3.24853246]
 step number 8error signed-2.4253938965393806 ,velyaw,-0.24253938965393806
[2.5188607  3.20465723]
 step number 9error signed-2.518860702092082 ,velyaw,-0.2518860702092082
[2.44324247 2.72426013]
 step number 10error signed-2.443242465284503 ,velyaw,-0.2443242465284503
[2.12338618 3.79636457]
 step number 11error signed-2.1233861789099744 ,velyaw,-0.21233861789099745
[2.202237   3.86345438]
 step number 12error signed-2.2022369968436672 ,velyaw,-0.22022369968436672
[2.28708994 3.92528119]
 step number 13error signed-2.2870899445451074 ,velyaw,-0.22870899445451076
[2.39402241 3.97116744]
 step number 14error signed-2.3940224095753226 ,velyaw,-0.23940224095753226
[2.49166147 4.01239871]
 step number 15error signed-2.4916614697090025 ,velyaw,-0.24916614697090025
[2.59700521 4.02475574]
 step number 16error signed-2.597005213954422 ,velyaw,-0.2597005213954422
[2.7017351

KeyboardInterrupt: 

In [ ]:
#############################################

In [ ]:
last_es=[]
for i in range(200):
    #xyz=get_coords()
    #closest_index=    np.linalg.norm((cc-xyz.T),axis=0).argmin()
    #closest_index_1= closest_index + 1
    #p1,p2= cc[closest_index,:2],cc[closest_index_1,:2]
    #p3= get_coords()[:2]
    #d = np.linalg.norm(np.cross(p2-p1, p1-p3))/np.linalg.norm(p2-p1)
    #error =d    
    error=get_signed_e()
    last_es.append(error)
    if len (last_es)==20:last_es.pop(0)
    aux_np= np.asarray(last_es)
    d_error=0
    if len (aux_np)>1:d_error= aux_np[-1]-aux_np[-2]
    int_T=aux_np.sum()
    print (int_T)
    
    #PID CONTROL
    # SINGLE STEP
    Kp=0.02
    Ki=  0.02
    Kd=   40
    vel_yaw= error * Kp  + int_T* Ki + d_error*Kd
    if (vel_yaw)>0.6:vel_yaw=0.6
    elif (vel_yaw)<-0.6:vel_yaw=-0.6
    print('error signed',error, 'velyaw',vel_yaw)
    #move_base_vel(0.15,0.0, vel_yaw)
    move_base(0.21,0.0, vel_yaw, 0.25)



In [ ]:
get_signed_e()

In [ ]:
get_signed_e()

In [ ]:
xyth= get_coords()
xyth

In [ ]:
#Orientation from robot to p1
pto_rob=xyz[:2]-p1
print(np.arctan2( pto_rob[1], pto_rob[0]))
#Get  cuadrant relative to robot to assign sign in error
th=xyth[-1]
np.rad2deg(th),th


In [ ]:
p1

In [ ]:
v31=(p3-p1)
v31
np.arctan2(v31[1],v31[0])

In [ ]:
v31

In [ ]:
v31=(p3-p1)
np.rad2deg(np.arctan2(v31[1],v31[0]))

In [ ]:
error =d    
#PROPORTIONAL CONTROL
# SINGLE STEP
Kp=1.0
vel_yaw= -error * Kp

move_base(0.05,0.0, vel_yaw, 1)



In [ ]:

move_base(0.1,0 ,0.5)

In [ ]:
xyz

In [ ]:
cc=np.asarray((cents_path))
xyz=np.expand_dims(xyz,axis=1)

In [ ]:
c=np.asarray((cents_path))
c

Nos valdremos de nuestra vieja función de movimiento, asi como la clase Laser para obtener las lecturas del sensor (revisar notebook 1)

Simultáneamente , es publicado en el tópico "/hsrb/head_rgbd_sensor/depth_registered/rectified_points"
, información de la nube de puntos, en forma de un mensaje tipo  PointCloud2

In [ ]:

def move_base_vel(vx, vy, vw):
    twist = Twist()
    twist.linear.x = vx
    twist.linear.y = vy
    twist.angular.z = vw 
    base_vel_pub.publish(twist)

def move_base(x,y,yaw,timeout=5):
    start_time = rospy.Time.now().to_sec()
    while rospy.Time.now().to_sec() - start_time < timeout:  
        move_base_vel(x, y, yaw)
    move_base_vel(0,0, 0)

In [ ]:
from sensor_msgs.msg import LaserScan

class Laser():
    u"""Class that handles laser information"""

    def __init__(self):
        # Register the _laser_cb method as a callback to the laser scan topic events
        self._laser_sub = rospy.Subscriber ('scan',
                                           LaserScan, self._laser_cb)
        self._scan_data = None

    def _laser_cb (self, msg):
        # Laser scan callback function
        self._scan_data = msg

    def get_data(self):
        u"""Function to get the laser value"""
        return self._scan_data

In [ ]:
laser = Laser()  #instanciamos una clase 

In [ ]:
xyz=get_coords()

In [ ]:
"""local_path=rospy.wait_for_message('/move_base/DWAPlannerROS/local_plan',Path)
global_path=rospy.wait_for_message('/move_base/DWAPlannerROS/global_plan',Path)"""

In [ ]:
global_path.header

In [ ]:
len(global_path.poses)
path= global_path


In [ ]:
pub_path.publish(path)